In [1]:
#dependencies

import pandas as pd
import numpy as np
import requests
import json

moviedbapi = "87594855563381b8f9fa3b3224c9f19f"
idURL = "https://api.themoviedb.org/3/search/movie?query="
idURL1 = "&api_key="
detailURL = "https://api.themoviedb.org/3/movie/"
detailURL1 = "?api_key="

In [2]:
omdbapi = "e8913e4c"
omdb1 = "http://www.omdbapi.com/?t="
omdb2 = "&apikey="

In [3]:
#import csv file as pd dataframe

csvPath = 'movies.csv'
opening = pd.read_csv(csvPath)
print(len(opening))
opening.head()

1500


,release,title,genre,budget,opening,domestic,worldwide
0,20170120,Split,Horror,5000000,40010975,138141585,278449984
1,20170120,xXx: Return of Xander Cage,Action,85000000,20130142,44898413,345114933
2,20170127,A Dog’s Purpose,Drama,25000000,18222810,64321890,205272612
3,20170106,Underworld: Blood Wars,Action,0,13688751,30405609,81096784
4,20170127,Resident Evil: The Final Chapter,Action,40000000,13601682,26844692,307353906


In [4]:
#filters out movies that made at least $10000 in the first weekend

opening = opening.loc[(opening['opening']>999)]
print(len(opening))
opening.head()

642


,release,title,genre,budget,opening,domestic,worldwide
0,20170120,Split,Horror,5000000,40010975,138141585,278449984
1,20170120,xXx: Return of Xander Cage,Action,85000000,20130142,44898413,345114933
2,20170127,A Dog’s Purpose,Drama,25000000,18222810,64321890,205272612
3,20170106,Underworld: Blood Wars,Action,0,13688751,30405609,81096784
4,20170127,Resident Evil: The Final Chapter,Action,40000000,13601682,26844692,307353906


In [5]:
#filters our rows with the budgets already populated and saves for later

budgetDF = opening.loc[(opening["budget"] > 0)]
print(len(budgetDF))
budgetDF.head()

132


,release,title,genre,budget,opening,domestic,worldwide
0,20170120,Split,Horror,5000000,40010975,138141585,278449984
1,20170120,xXx: Return of Xander Cage,Action,85000000,20130142,44898413,345114933
2,20170127,A Dog’s Purpose,Drama,25000000,18222810,64321890,205272612
4,20170127,Resident Evil: The Final Chapter,Action,40000000,13601682,26844692,307353906
5,20170113,The Bye Bye Man,Thriller/Suspense,7400000,13501349,22395806,25863405


In [6]:
nobudgetDF = opening.loc[(opening["budget"] == 0)]
print(len(nobudgetDF))
nobudgetDF.head()

510


,release,title,genre,budget,opening,domestic,worldwide
3,20170106,Underworld: Blood Wars,Action,0,13688751,30405609,81096784
7,20170113,Sleepless,Thriller/Suspense,0,8344128,20783704,27224309
8,20170127,Gold,Drama,0,3471316,7227038,11478501
10,20170125,Raees,Action,0,1798379,2369469,5429008
11,20170120,The Resurrection of Gavin Stone,Comedy,0,1206771,2303792,2303792


In [7]:
#get ids
idList = []
df = []

for index, row in nobudgetDF.iterrows():
#     print('--------------------------------------------------------------------------------')
    movie = row['title']
#     finalMovie = movie.replace(":","")
#     finalMovie = movie.replace(" ","%20")
    openGross = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    date = row['release']
    genre = row['genre']
    finalURL = idURL + movie + idURL1 + moviedbapi
#     print (finalURL)
    json_file = requests.get(finalURL).json()
    moviesJSON = json_file['results']
#     print (moviesJSON)
#     print (len(json_file['results']))
    
    try:
        dates = []
        for n in np.arange(len(moviesJSON)):
#             print (n)
            dt = moviesJSON[n]['release_date']
            dt = int(dt.replace("-",""))
#             print (dt)
            dates.append(dt)
#             print (f"success {n}")
#         print (dates)      
#         print (recent)
    except:
        pass
#         print ("no dates")
    try:
        for number in dates:
#             print (number)
            if date-30 <= number and number <= date + 30:
#                 print("DateMatch")
                ind = dates.index(number)
#                 print (ind)
#                 print (moviesJSON)
                recentMovie = moviesJSON[ind]
                idNumber = recentMovie['id']
#                 print (f"success {n}.{n}")
#                 movie = recentMovie['original_title']
#                 print(f'{movie} / {number} / {ind} / {idNumber} / {openGross}')
                movieDict = {"release":number,"title":movie, "gross":openGross, "id_number":idNumber, "domestic":domestic, 
                             "worldwide":worldwide, "genre":genre}
                df.append(movieDict)
            else:
                continue
    
    except:
        print ("no id")
    
idDF = pd.DataFrame(df)
print (len(idDF))
pd.options.display.max_rows = 999
idDF.head()

237


,domestic,genre,gross,id_number,release,title,worldwide
0,20783704,Thriller/Suspense,8344128,324542,20170112,Sleepless,27224309
1,2369469,Action,1798379,363343,20170125,Raees,5429008
2,2303792,Comedy,1206771,422114,20170120,The Resurrection of Gavin Stone,2303792
3,355922,Drama,225968,413882,20170112,Ok Jaanu,3250846
4,292662,Action,181705,435863,20170127,Buddies in India,108878601


In [8]:
#get budgets
budgets = []
for index, row in idDF.iterrows():
    genre = row['genre']
    gross = row['gross']
    title = row['title']
    domestic = row['domestic']
    worldwide = row['worldwide']

    try:
        
        detailsURL = detailURL + str(row['id_number']) + detailURL1 + moviedbapi
#     print (detailsURL)
        details = requests.get(detailsURL).json()
        budget = details['budget']
        release = details['release_date']
#         print (f"Budget Found = {budget}")
        fullDict = {"title": title, "opening":gross, "budget":budget, "domestic":domestic, 
                    "genre":genre, "release":release, "worldwide":worldwide}
        budgets.append(fullDict)
    except:
        pass
#     print (genre)
    
addbudgetDF = pd.DataFrame(budgets)
addbudgetDF = addbudgetDF.loc[(addbudgetDF['budget']>0)]
print(len(addbudgetDF))
addbudgetDF.head()

56


,budget,domestic,genre,opening,release,title,worldwide
0,30000000,20783704,Thriller/Suspense,8344128,2017-01-12,Sleepless,27224309
1,9645980,2369469,Action,1798379,2017-01-25,Raees,5429008
2,2000000,2303792,Comedy,1206771,2017-01-20,The Resurrection of Gavin Stone,2303792
5,65000000,362657,Action,111979,2017-01-27,Gong fu yu jia,254569616
8,40000000,8106986,Thriller/Suspense,4356941,2017-02-15,A Cure for Wellness,24499901


In [10]:
# add budget to original dataframe
# release	title	genre	budget	opening	domestic	worldwide
budgetFinal = []

for index, row in budgetDF.iterrows():
    release = row['release']
    title = row['title']
    genre = row['genre']
    budget = row['budget']
    opening = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    moviesData = {"release": release, "title":title, "genre":genre, "budget":budget, 
                  "opening":opening, "domestic":domestic, "worldwide":worldwide}
    budgetFinal.append(moviesData)
    
for index, row in addbudgetDF.iterrows():
    release = row['release']
    release = release.replace('-','')
    title = row['title']
    genre = row['genre']
    budget = row['budget']
    opening = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    moviesData = {"release": release, "title":title, "genre":genre, "budget":budget, 
                  "opening":opening, "domestic":domestic, "worldwide":worldwide}
    budgetFinal.append(moviesData)
    
moviesDF = pd.DataFrame(budgetFinal)
print (len(moviesDF))
moviesDF.head()

188


,budget,domestic,genre,opening,release,title,worldwide
0,5000000,138141585,Horror,40010975,20170120,Split,278449984
1,85000000,44898413,Action,20130142,20170120,xXx: Return of Xander Cage,345114933
2,25000000,64321890,Drama,18222810,20170127,A Dog’s Purpose,205272612
3,40000000,26844692,Action,13601682,20170127,Resident Evil: The Final Chapter,307353906
4,7400000,22395806,Thriller/Suspense,13501349,20170113,The Bye Bye Man,25863405


In [37]:
# get ratings
moviesFinal =[]
for index, row in moviesDF.iterrows():
    release = row['release']
    title = row['title']
    genre = row['genre']
    budget = row['budget']
    opening = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    rbratio = opening/budget
    omdbURL = omdb1 + title + omdb2 + omdbapi
#     print (omdbURL)
    ratingJSON = requests.get(omdbURL).json()
    ratings = []
    finalRatings = []
    try:
        
        ratingJSON = ratingJSON['Ratings']
#         print (ratingJSON)
        for x in np.arange(len(ratingJSON)):
            value = ratingJSON[x]['Value']
            ratings.append(value)
#         print (ratings)
        for number in ratings:
#             print (number)
#             print (number.replace("'",""))
            numbers = number.replace("'","")
#             print (numbers)
            numbers = numbers.replace("/100","")
            numbers = numbers.replace("/10","")
            numbers = numbers.replace(".","")
            numbers = numbers.replace("%","")
            finalRatings.append(int(numbers))
    except:
        finalRatings = 0
    try:
        avgRating = np.mean(finalRatings).round(2)
#         print (avgRating)
    except:
        avgRating = 0
    
    moviesFinalDict = {"release": release, "title":title, "genre":genre, "budget":budget, 
                      "opening":opening, "ratio":rbratio, "domestic":domestic, 
                       "worldwide":worldwide, "value": avgRating}
    moviesFinal.append(moviesFinalDict)
moviesFinalDF = pd.DataFrame(moviesFinal)

moviesFinalDF = moviesFinalDF[['title','genre','release','value','budget','opening','domestic','worldwide', 'ratio']]
moviesFinalDF = moviesFinalDF.dropna(how='any')

moviesFinalDF.to_csv('moviesFinal.csv', encoding='utf-8', index=False)
print (len(moviesFinalDF))
moviesFinalDF.head()

C:\Users\mcyee\Anaconda3\envs\pythondata\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\mcyee\Anaconda3\envs\pythondata\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


186


,title,genre,release,value,budget,opening,domestic,worldwide,ratio
0,Split,Horror,20170120,70.00,5000000,40010975,138141585,278449984,8.002195
1,xXx: Return of Xander Cage,Action,20170120,46.33,85000000,20130142,44898413,345114933,0.236825
2,A Dog’s Purpose,Drama,20170127,48.33,25000000,18222810,64321890,205272612,0.728912
3,Resident Evil: The Final Chapter,Action,20170127,47.00,40000000,13601682,26844692,307353906,0.340042
4,The Bye Bye Man,Thriller/Suspense,20170113,34.33,7400000,13501349,22395806,25863405,1.824507


In [38]:
moviesFinalDF['zrating'] = ''
moviesFinalDF['zbudget'] = ''
moviesFinalDF['zopening'] = ''
moviesFinalDF['ztotal'] = ''
ratingsMean = np.mean(moviesFinalDF['value'])
ratingsSD = np.std(moviesFinalDF['value'])
budgetMean = np.mean(moviesFinalDF['budget'])
budgetSD = np.std(moviesFinalDF['budget'])
openingMean = np.mean(moviesFinalDF['opening'])
openingSD = np.std(moviesFinalDF['opening'])

for index, movie in moviesFinalDF.iterrows():
    zrat = (movie['value'] - ratingsMean)/ratingsSD
    moviesFinalDF.set_value(index, 'zrating', zrat)
    zbud = (movie['budget'] - budgetMean)/budgetSD
    moviesFinalDF.set_value(index, 'zbudget', zbud)
    zope = (movie['opening'] - openingMean)/openingSD
    moviesFinalDF.set_value(index, 'zopening', zope)
    ztot = zrat + zbud + (2 * zope)
    moviesFinalDF.set_value(index, 'ztotal', ztot)

moviesFinalDF = moviesFinalDF.sort_values(by =['ztotal'], ascending = False)
print (len(moviesFinalDF))
moviesFinalDF

186


,title,genre,release,value,budget,opening,domestic,worldwide,ratio,zrating,zbudget,zopening,ztotal
119,Black Panther,Action,20180216,88.00,200000000,202003951,561697180,1078297180,1.010020,1.42906,2.52071,5.33172,14.6132
20,Beauty and the Beast,Musical,20170317,89.67,160000000,174750616,504014165,1263109573,1.092191,1.50499,1.84422,4.533,12.4152
174,Star Wars Ep. VIII: The Last Jedi,Action,20171213,0.00,200000000,220009584,619483244,1321513738,1.100048,-2.57232,2.52071,5.85942,11.6672
41,Guardians of the Galaxy Vol 2,Action,20170505,75.67,200000000,146510104,389813101,863197744,0.732551,0.86841,2.52071,3.70534,10.7998
95,Thor: Ragnarok,Action,20171103,82.00,180000000,122744989,315034967,850630090,0.681917,1.15624,2.18246,3.00885,9.3564
60,Spider-Man: Homecoming,Action,20170707,80.00,175000000,117027503,334201140,880206511,0.668729,1.0653,2.0979,2.84128,8.84577
96,Justice League,Action,20171117,51.00,300000000,93842239,229007315,655936466,0.312807,-0.25334,4.21193,2.16178,8.28216
32,The Fate of the Furious,Action,20170414,63.00,250000000,98786705,225764765,1237466026,0.395147,0.292303,3.36632,2.30669,8.27201
49,Wonder Woman,Action,20170602,81.00,150000000,103251471,412563408,817012815,0.688343,1.11077,1.6751,2.43754,7.66095
78,It,Horror,20170908,76.33,35000000,123403419,327481748,697459228,3.525812,0.898421,-0.269809,3.02814,6.6849
